# Benchmark analysis with Cycle 2 Output

In [1]:
import pathlib
import gribscan
import intake
import xarray as xr
import dask
import pandas as pd
from dask_jobqueue import SLURMCluster
from dask.distributed import Client
dask.config.config.get('distributed').get('dashboard').update({'link':'{JUPYTERHUB_SERVICE_PREFIX}/proxy/{port}/status'})

/work/ab0995/AWIsoft/miniconda/NextGEMS/.conda/lib/python3.9/site-packages/dask_jobqueue/core.py:20: FutureWarning: tmpfile is deprecated and will be removed in a future release. Please use dask.utils.tmpfile instead.
  from distributed.utils import tmpfile


In [7]:
# FIXME(PG): These numbers still need some tuning, they are not yet optimized...
# NOTE(PG): Using 300Gb and 8 Cores leads to a non-crashing Dask Cluster, but probably more cores can be given.
cluster = SLURMCluster(
    #name='dask-cluster', 
    #processes=24,
    cores=8,
    # memory=f"{8000 * 64 * 0.90} MB",
    memory="300Gb",
    project="ab0995",
    queue="compute",
    interface='ib0',
    walltime='08:00:00',
    asynchronous=0,
    job_extra=["-o dask-worker-%j.log", "-e dask-worker-%j.err", "--verbose"],
)

print(cluster.job_script())

#!/usr/bin/env bash

#SBATCH -J dask-worker
#SBATCH -p compute
#SBATCH -A ab0995
#SBATCH -n 1
#SBATCH --cpus-per-task=8
#SBATCH --mem=280G
#SBATCH -t 08:00:00
#SBATCH -o dask-worker-%j.log
#SBATCH -e dask-worker-%j.err
#SBATCH --verbose

/work/ab0995/AWIsoft/miniconda/NextGEMS/.conda/bin/python -m distributed.cli.dask_worker tcp://136.172.115.19:41031 --nthreads 2 --nprocs 4 --memory-limit 69.85GiB --name dummy-name --nanny --death-timeout 60 --interface ib0 --protocol tcp://



/work/ab0995/AWIsoft/miniconda/NextGEMS/.conda/lib/python3.9/site-packages/distributed/node.py:177: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 42791 instead
  warnings.warn(


In [8]:
# Adaptive scaling. This requests number of **workers** (NOT slurm compute nodes!!)
cluster.adapt(minimum=10, maximum=1000)

In [9]:
client = Client(cluster)

In [10]:
# Show the client object to get a sense for what is there.
# As workers are added or removed, the dashboard might hang!
client

<Client: 'tcp://136.172.115.19:41031' processes=12 threads=24, memory=838.20 GiB>

In [11]:
catalog_file = "/work/mh0287/m221078/intake/sop_scrum.json"

In [12]:
!head -20 "/work/mh0287/m221078/intake/sop_scrum.json"

{
    "esmcat_version": "0.1.0",
    "id": "ICON-ESM",
    "description": "Default catalog for ICON-ESM experiments by MPIMet",
    "attributes": [
        {
            "column_name": "variable_id"
        },
        {
            "column_name": "project"
        },
        {
            "column_name": "institution_id"
        },
        {
            "column_name": "source_id"
        },
        {
            "column_name": "experiment_id"
        },


In [13]:
cat = intake.open_esm_datastore(catalog_file)
cat

/work/ab0995/AWIsoft/miniconda/NextGEMS/.conda/lib/python3.9/site-packages/fastprogress/fastprogress.py:102: UserWarning: Couldn't import ipywidgets properly, progress bar will use console behavior
  warn("Couldn't import ipywidgets properly, progress bar will use console behavior")


,unique
variable_id,155
project,1
institution_id,1
source_id,1
experiment_id,1
simulation_id,2
realm,2
frequency,4
time_reduction,2
grid_label,1


In [14]:
cat.df.head(n=2)

,variable_id,project,institution_id,source_id,experiment_id,simulation_id,realm,frequency,time_reduction,grid_label,level_type,time_min,time_max,grid_id,format,uri
0,"(ps, psl, rsdt, rsut, rsutcs, rlut, rlutcs, rs...",private,MPI-M,ICON-ESM,dpp,dpp0067,atm,30minute,mean,gn,ml,2020-01-20T00:00:00.000,2020-01-20T23:59:20.000,not implemented,netcdf,/work/mh0287/m218027/experiments/dpp0067/dpp00...
1,"(ps, psl, rsdt, rsut, rsutcs, rlut, rlutcs, rs...",private,MPI-M,ICON-ESM,dpp,dpp0067,atm,30minute,mean,gn,ml,2020-01-21T00:00:00.000,2020-01-21T23:59:20.000,not implemented,netcdf,/work/mh0287/m218027/experiments/dpp0067/dpp00...


In [15]:
cat.unique("frequency")

{'frequency': {'count': 4, 'values': ['30minute', '3hour', '1hour', '1day']}}

In [16]:
entry = "frequency"
cat.unique(entry)[entry]["values"]

['30minute', '3hour', '1hour', '1day']

In [17]:
hits_0 = cat.search(variable_id="tas")
hits_0

,unique
variable_id,37
project,1
institution_id,1
source_id,1
experiment_id,1
simulation_id,2
realm,1
frequency,1
time_reduction,1
grid_label,1


In [18]:
entry = "simulation_id"
hits_0.unique(entry)[entry]["values"]

['dpp0067', 'dpp0066']

In [19]:
hits = cat.search(variable_id="tas", simulation_id="dpp0066")
hits

,unique
variable_id,37
project,1
institution_id,1
source_id,1
experiment_id,1
simulation_id,1
realm,1
frequency,1
time_reduction,1
grid_label,1


In [22]:
%%time
dataset_dict = hits.to_dataset_dict(
    cdf_kwargs={
        "chunks": dict(
            height=1,
        )
    }
)


--> The keys in the returned dictionary of datasets are constructed as follows:
	'project.institution_id.source_id.experiment_id.simulation_id.realm.frequency.time_reduction.grid_label.level_type'
CPU times: user 1.52 s, sys: 112 ms, total: 1.63 s
Wall time: 3.76 s


In [23]:
keys = list(dataset_dict.keys())
keys

['private.MPI-M.ICON-ESM.dpp.dpp0066.atm.30minute.mean.gn.ml']

In [24]:
dataset = dataset_dict[keys[0]]
dataset

<xarray.Dataset>
Dimensions:   (time: 19488, height_2: 1, ncells: 20971520)
Coordinates:
  * height_2  (height_2) float64 2.0
  * time      (time) float64 2.02e+07 2.02e+07 2.02e+07 ... 2.021e+07 2.021e+07
Dimensions without coordinates: ncells
Data variables:
    tas       (time, height_2, ncells) float32 dask.array<chunksize=(48, 1, 20971520), meta=np.ndarray>
Attributes: (12/13)
    grid_file_uri:           http://icon-downloads.mpimet.mpg.de/grids/public...
    intake_esm_varname:      ['tas']
    institution:             Max Planck Institute for Meteorology/Deutscher W...
    references:              see MPIM/DWD publications
    uuidOfHGrid:             0f1e7d66-637e-11e8-913b-51232bb4d8f9
    CDI:                     Climate Data Interface version 1.8.3rc (http://m...
    ...                      ...
    number_of_grid_used:     15
    source:                  icon-dkrz\tgit@gitlab.dkrz.de:icon/icon-dkrz.git...
    title:                   ICON simulation
    Conventions:             CF-1.6
    history:                 ./icon at 20220218 173036\n./icon at 20220218 20...
    intake_esm_dataset_key:  private.MPI-M.ICON-ESM.dpp.dpp0066.atm.30minute....

In [25]:
grid_file_path = "/pool/data/ICON" + dataset.grid_file_uri.split(".de")[1]
grid_file_path

'/pool/data/ICON/grids/public/mpim/0015/icon_grid_0015_R02B09_G.nc'

In [26]:
grid_data = xr.open_dataset(grid_file_path).rename(
    cell="ncells"
)  # the dimension has different names in the grid file and in the output.

In [27]:
data = xr.merge((dataset, grid_data))
data

<xarray.Dataset>
Dimensions:                         (time: 19488, height_2: 1,
                                     ncells: 20971520, nv: 3, vertex: 10485762,
                                     ne: 6, edge: 31457280, no: 4, nc: 2,
                                     max_stored_decompositions: 4, two_grf: 2,
                                     cell_grf: 14, max_chdom: 1, edge_grf: 24,
                                     vert_grf: 13)
Coordinates:
  * height_2                        (height_2) float64 2.0
  * time                            (time) float64 2.02e+07 ... 2.021e+07
    clon                            (ncells) float64 ...
    clat                            (ncells) float64 ...
    vlon                            (vertex) float64 ...
    vlat                            (vertex) float64 ...
    elon                            (edge) float64 ...
    elat                            (edge) float64 ...
Dimensions without coordinates: ncells, nv, vertex, ne, edge, no, nc,
                                max_stored_decompositions, two_grf, cell_grf,
                                max_chdom, edge_grf, vert_grf
Data variables: (12/92)
    tas                             (time, height_2, ncells) float32 dask.array<chunksize=(48, 1, 20971520), meta=np.ndarray>
    clon_vertices                   (ncells, nv) float64 ...
    clat_vertices                   (ncells, nv) float64 ...
    vlon_vertices                   (vertex, ne) float64 ...
    vlat_vertices                   (vertex, ne) float64 ...
    elon_vertices                   (edge, no) float64 ...
    ...                              ...
    edge_dual_normal_cartesian_x    (edge) float64 ...
    edge_dual_normal_cartesian_y    (edge) float64 ...
    edge_dual_normal_cartesian_z    (edge) float64 ...
    cell_circumcenter_cartesian_x   (ncells) float64 ...
    cell_circumcenter_cartesian_y   (ncells) float64 ...
    cell_circumcenter_cartesian_z   (ncells) float64 ...
Attributes: (12/13)
    grid_file_uri:           http://icon-downloads.mpimet.mpg.de/grids/public...
    intake_esm_varname:      ['tas']
    institution:             Max Planck Institute for Meteorology/Deutscher W...
    references:              see MPIM/DWD publications
    uuidOfHGrid:             0f1e7d66-637e-11e8-913b-51232bb4d8f9
    CDI:                     Climate Data Interface version 1.8.3rc (http://m...
    ...                      ...
    number_of_grid_used:     15
    source:                  icon-dkrz\tgit@gitlab.dkrz.de:icon/icon-dkrz.git...
    title:                   ICON simulation
    Conventions:             CF-1.6
    history:                 ./icon at 20220218 173036\n./icon at 20220218 20...
    intake_esm_dataset_key:  private.MPI-M.ICON-ESM.dpp.dpp0066.atm.30minute....

In [28]:
idx = pd.date_range(start="2020-01-20", freq="1D", periods=len(data.tas.time))
data['time'] = idx
year_month_idx = pd.MultiIndex.from_arrays([data['time.year'].data, data['time.month'].data])
data.coords['year_month'] = ('time', year_month_idx)

In [29]:
with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    tas_year_months = data.tas.groupby('year_month').max(dim="time")

In [30]:
tas_time_max = tas_year_months.max(dim="year_month")

In [31]:
tas_time_max

<xarray.DataArray 'tas' (height_2: 1, ncells: 20971520)>
dask.array<_nanmax_skip-aggregate, shape=(1, 20971520), dtype=float32, chunksize=(1, 20971520), chunktype=numpy.ndarray>
Coordinates:
  * height_2  (height_2) float64 2.0
    clon      (ncells) float64 ...
    clat      (ncells) float64 ...
Dimensions without coordinates: ncells

In [32]:
with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    tas_time_max_binned = tas_year_months.groupby_bins("clon", 160).max(dim="year_month")

/work/ab0995/AWIsoft/miniconda/NextGEMS/.conda/lib/python3.9/site-packages/xarray/core/indexing.py:1228: PerformanceWarning: Slicing with an out-of-order index is generating 1535 times more chunks
  return self.array[key]


In [33]:
tas_time_max_binned

<xarray.DataArray 'tas' (height_2: 1, ncells: 20971520)>
dask.array<getitem, shape=(1, 20971520), dtype=float32, chunksize=(1, 2080), chunktype=numpy.ndarray>
Coordinates:
  * height_2  (height_2) float64 2.0
    clon      (ncells) float64 1.274 1.275 1.273 1.274 ... 1.335 1.336 1.334
    clat      (ncells) float64 ...
Dimensions without coordinates: ncells

In [ ]:
tas_time_max = tas_time_max.compute()

Task exception was never retrieved
future: <Task finished name='Task-37828' coro=<WebSocketProtocol13.write_message.<locals>.wrapper() done, defined at /work/ab0995/AWIsoft/miniconda/NextGEMS/.conda/lib/python3.9/site-packages/tornado/websocket.py:1100> exception=WebSocketClosedError()>
Traceback (most recent call last):
  File "/work/ab0995/AWIsoft/miniconda/NextGEMS/.conda/lib/python3.9/site-packages/tornado/websocket.py", line 1102, in wrapper
    await fut
tornado.iostream.StreamClosedError: Stream is closed

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/work/ab0995/AWIsoft/miniconda/NextGEMS/.conda/lib/python3.9/site-packages/tornado/websocket.py", line 1104, in wrapper
    raise WebSocketClosedError()
tornado.websocket.WebSocketClosedError
Task exception was never retrieved
future: <Task finished name='Task-42049' coro=<WebSocketProtocol13.write_message.<locals>.wrapper() done, defined at /work/ab0995/AWIsoft/mini

In [ ]:
tas_time_max_binned = tas_time_max_binned.compute()